# QUESTION 2

## Import dataset and data cleaning

In [274]:
import pandas as pd
from datetime import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [275]:
df=pd.read_csv(r"C:\Users\gianl\Documents\Università_4°anno(2022-2023\ADM\HOMEWORK\HOMEWORK 4\bank_transactions.csv", sep=",")

In [276]:
#traformiamo le variaibli in datetime
df['TransactionTime'] = df['TransactionTime'].apply(lambda x: datetime.strptime(str(x).zfill(6), '%H%M%S'))
df['CustomerDOB'] = pd.to_datetime(df['CustomerDOB'], format = '%d/%m/%y', errors = 'coerce')
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], format = '%d/%m/%y', errors = 'coerce')
#da transaction time vogliamo levare la data, applichiamo questo comando quidi per avere solo l'oario
#df['TransactionTime'] = df['TransactionTime'].apply( lambda d : d.time() ) 

In [249]:
df.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,1994-01-10,F,JAMSHEDPUR,17819.05,2016-08-02,1900-01-01 14:32:07,25.00
1,T2,C2142763,2057-04-04,M,JHAJJAR,2270.69,2016-08-02,1900-01-01 14:18:58,27999.00
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-08-02,1900-01-01 14:27:12,459.00
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-08-02,1900-01-01 14:27:14,2060.00
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-08-02,1900-01-01 18:11:56,1762.50


In [229]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048567 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   TransactionID            1048567 non-null  object        
 1   CustomerID               1048567 non-null  object        
 2   CustomerDOB              987831 non-null   datetime64[ns]
 3   CustGender               1047467 non-null  object        
 4   CustLocation             1048416 non-null  object        
 5   CustAccountBalance       1046198 non-null  float64       
 6   TransactionDate          1048567 non-null  datetime64[ns]
 7   TransactionTime          1048567 non-null  object        
 8   TransactionAmount (INR)  1048567 non-null  float64       
dtypes: datetime64[ns](2), float64(2), object(5)
memory usage: 72.0+ MB


In [ ]:
a = df.groupby('CustomerID')

In [ ]:
#for i in a:
#    print(i)

**There are some values missing but very few compering to the rest of the dataset, so we can drop it**

In [277]:
df = df.dropna()

**As we can see, there are a lot of birthdates on January 1st, which seems a little strange. There's also a lot of customers birth on 1/1/1800. This is probably some default the bank has where this information is unknown, althought that is a little strange. I'll remove them.**

In [180]:
df['CustomerDOB'].value_counts()

1989-01-01    809
1990-01-01    784
1991-08-06    698
1991-01-01    665
1992-01-01    631
             ... 
2005-02-23      1
2042-11-28      1
2049-09-23      1
2040-03-14      1
2044-10-24      1
Name: CustomerDOB, Length: 17232, dtype: int64

In [278]:
df = df.loc[~(df['CustomerDOB'] == '1/1/1800')]

**There are people born after 2021, so just subtract 100 from the year of birth of these people**

In [279]:
df.loc[df['CustomerDOB'].dt.year >= 2021, ['CustomerDOB']] -= pd.DateOffset(years = 100)

# 2.1

In [280]:
#Let's create a new dataframe called new_df
new_df = pd.DataFrame()

**Number of transactions**

In [281]:
new_df['NumTransaction'] = df.groupby(['CustomerID'])['TransactionID'].count()

**Number of transactions > 100**

For each customer
* find transactions > 100
* count number of rows for each customer

In [282]:
new_df['AmountTrans>100'] = df[df['TransactionAmount (INR)']  > 100].groupby(["CustomerID"])["TransactionAmount (INR)"].count()
new_df['AmountTrans>100'] = new_df['AmountTrans>100'].fillna(0)
new_df['AmountTrans>100'] = new_df['AmountTrans>100'].astype(int)

**Average amount of transaction**

In [283]:
new_df['AvgAmountTransaction'] = df.groupby(['CustomerID'])['TransactionAmount (INR)'].agg('mean')

**Average balance**

In [284]:
#The first command is to visualize a float number approximated to two decimal places and not an exponential number.
pd.options.display.float_format = '{:.2f}'.format 
new_df['AvgBalance'] = df.groupby(['CustomerID'])['CustAccountBalance'].agg('mean')

**Average difference between the balance and the transaction amount for each customer**

In [285]:
df['provvisoria'] = df['CustAccountBalance'] - df['TransactionAmount (INR)']
new_df['AvgDiffBalTrans'] = df.groupby(['CustomerID'])['provvisoria'].agg('mean')
del df['provvisoria']

**Gender of customers???**

In [307]:
Gender = df.groupby(['CustomerID'])['CustGender'].first()
new_df['Gender'] = Gender

In [204]:
new_df.groupby('Gender').count()

,NumTransaction,AvgAmountTransaction,AvgBalance,AvgDiffBalTrans
Gender,,,,
F,273256,367447135.92,23877167001.38,23509719865.46
M,712066,852260783.19,64943365789.12,64091105005.93


**Most frequent location of the customer???**

In [13]:
new_df['MostFreqLocation'] = df.groupby(['CustomerID'])['CustLocation'].agg(pd.Series.mode)

## Create 20 additional features

The addictional futures that we will cosider are:

1. Highest transiction amount
2. Lowest transiction amount
3. Highest balance a customer has ever had
4. Lowest balance a customer has ever had
5. Number of transaction carried out just during August
6. Number of transaction carried out just during September
7. Number of transaction carried out just during October
8. Age of customers on transition days
9. Number of transactions > 1000
10. Number of transactions > 5000
11. Number of transactions > 10000
12. Number of times a transition has been made in the most frequent location of the customer
13. Number of transitions made in the night (22-06)
14. Number of transitions made in the morning (06-14)
15. Number of transitions made in the evening (14-22)
16. Percentile of the transaction amount, taking into account all transaction amounts
17. Percentile of the account balance, taking into account all account balances
18. Time between birthday in 2016 and the transaction.
19. Average time between transactions. Comment: probably won't be that good because most people just have one transaction. But we will remove some features later anyways.
20. Season of birthday (Winter, Spring, Summer, Autumn). This is similar information to the birthday, but with less granularity.

Comment: Maybe we can use percentage in some places instead of number just to have some variety? E.g. percentage of transactions made at night?

**Notation regarding variables 5,6,7**

With regard to variables 5, 6 and 7, at the beginning we thought to see the number of transactions carried out during the 4 different seasons, but later we noticed that the transactions within the dataset were all carried out in the months of August, September and October. For this reason we have decided to create 3 new variables that tell us how many transactions users have made in the different months (August, September and October).

The following cell shows us that the transactions were actually carried out only in the 3 months indicated above.

In [76]:
df.groupby(df['TransactionDate'].dt.month).count()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
TransactionDate,,,,,,,,,
8,613605,613605,613605,613605,613605,613605,613605,613605,613605
9,368307,368307,368307,368307,368307,368307,368307,368307,368307
10,3410,3410,3410,3410,3410,3410,3410,3410,3410


**Notation regarding variables 9,10,11**

In [192]:
#I WOULD LIKE TO MAKE A RANGE BAR PLOT, THAT IS TO COUNT THE NUMBER OF TRANSACTIONS BETWEEN 100 AND 500, THEN THE ONE BETWEEN 500 AND 2000, 
#THE ONE BETWEEN 2000 AND 10000 ETC AND I WOULD LIKE TO SHOW THAT THERE ARE ONLY 47K TRANSACTIONS> OF 5000 SO I DECIDED TO CREATE THE VARIABLES> 500,
#> 1000 and> 5000. THAT MEANS, I TAKEN INTERVALS NOT THAT GREAT BECAUSE OTHERWISE HAD LITTLE SENSE.
#FOR NOW I CAN SHOW WHAT I SAID WITH A COMMAND BUT IT WAS NICE TO MAKE A GRAPH THAT SHOWED IT BUT I DON'T KNOW HOW TO DO IT
#df[df['TransactionAmount (INR)'] > 500].count()
df[df['TransactionAmount (INR)'] > 1000].count()
#df[df['TransactionAmount (INR)'] > 2000].count()
#df[df['TransactionAmount (INR)'] > 5000].count()

TransactionID              267271
CustomerID                 267271
CustomerDOB                267271
CustGender                 267271
CustLocation               267271
CustAccountBalance         267271
TransactionDate            267271
TransactionTime            267271
TransactionAmount (INR)    267271
dtype: int64

#### Once the additional features considered have been defined, we create them and add them to the 'new_df' dataset.

**1** Highest transiction amount

In [286]:
new_df['MaxAmountTransaction'] = df.groupby(['CustomerID'])['TransactionAmount (INR)'].agg('max')

**2** Lowest transiction amount

In [287]:
new_df['MinAmountTransaction'] = df.groupby(['CustomerID'])['TransactionAmount (INR)'].agg('min')

**3** Highest balance a customer has ever had

In [288]:
new_df['MaxBalance'] = df.groupby(['CustomerID'])['CustAccountBalance'].agg('max')

**4** Lowest balance a customer has ever had

In [289]:
new_df['MinBalance'] = df.groupby(['CustomerID'])['CustAccountBalance'].agg('min')

**5** Number of transaction carried out just during August

In [290]:
in_august = df[df['TransactionDate'].dt.month == 8]
new_df['TransactionAugust'] = in_august.groupby(['CustomerID'])['TransactionDate'].count()
new_df['TransactionAugust'] = new_df['TransactionAugust'].fillna(0) #we replace NaN values, that is who didn't make transaction in summer, with 0.
new_df['Transactio nAugust'] = new_df['TransactionAugust'].astype(int)

**6** Number of transaction carried out just during September

In [291]:
in_september = df[df['TransactionDate'].dt.month == 9]
new_df['TransactionSeptember'] = in_september.groupby(['CustomerID'])['TransactionDate'].count()
new_df['TransactionSeptember'] = new_df['TransactionSeptember'].fillna(0) #we replace NaN values, that is who didn't make transaction in summer, with 0.
new_df['TransactionSeptember'] = new_df['TransactionSeptember'].astype(int)

**7** Number of transaction carried out just during October

In [292]:
in_october = df[df['TransactionDate'].dt.month == 10]
new_df['TransactionOctober'] = in_october.groupby(['CustomerID'])['TransactionDate'].count()
new_df['TransactionOctober'] = new_df['TransactionOctober'].fillna(0) #we replace NaN values, that is who didn't make transaction in summer, with 0.
new_df['TransactionOctober'] = new_df['TransactionOctober'].astype(int)

**8** Age of customers on transition days

In [293]:
ex = df.groupby(['CustomerID'])[['TransactionDate','CustomerDOB']].first()
new_df['Age'] = ex['TransactionDate'].dt.year - ex['CustomerDOB'].dt.year

**9** Number of transactions > 500

In [294]:
new_df['AmountTrans>1000'] = df[df['TransactionAmount (INR)']  > 500].groupby(["CustomerID"])["TransactionAmount (INR)"].count()
new_df['AmountTrans>1000'] = new_df['AmountTrans>1000'].fillna(0)
new_df['AmountTrans>1000'] = new_df['AmountTrans>1000'].astype(int)

**10** Number of transactions > 1000

In [295]:
new_df['AmountTrans>5000'] = df[df['TransactionAmount (INR)']  > 1000].groupby(["CustomerID"])["TransactionAmount (INR)"].count()
new_df['AmountTrans>5000'] = new_df['AmountTrans>5000'].fillna(0)
new_df['AmountTrans>5000'] = new_df['AmountTrans>5000'].astype(int)

**11** Number of transactions > 5000

In [296]:
new_df['AmountTrans>10000'] = df[df['TransactionAmount (INR)']  > 5000].groupby(["CustomerID"])["TransactionAmount (INR)"].count()
new_df['AmountTrans>10000'] = new_df['AmountTrans>10000'].fillna(0)
new_df['AmountTrans>10000'] = new_df['AmountTrans>10000'].astype(int)

**12** Number of times a transition has been made in the most frequent location of the customer

**13** Number of transitions made in the night (23-06)

In [297]:
after_twentythree = df[df['TransactionTime'].dt.hour >= 23]
before_six = df[df['TransactionTime'].dt.hour <= 6]
between_twentythree_six = pd.concat([after_twentythree,before_six], ignore_index = True)
new_df['Transaction_23-06'] = between_twentythree_six.groupby(['CustomerID'])['TransactionTime'].count()
new_df['Transaction_23-06'] = new_df['Transaction_23-06'].fillna(0) #we replace NaN values, that is who didn't make transaction in summer, with 0.
new_df['Transaction_23-06'] = new_df['Transaction_23-06'].astype(int)

**14** Number of transitions made in the morning (07-14)

In [298]:
after_seven = df[df['TransactionTime'].dt.hour >= 7]
between_seven_forteen = after_seven[after_seven['TransactionTime'].dt.hour <= 14]
new_df['Transaction_7-14'] = between_seven_forteen.groupby(['CustomerID'])['TransactionTime'].count()
new_df['Transaction_7-14'] = new_df['Transaction_7-14'].fillna(0) #we replace NaN values, that is who didn't make transaction in summer, with 0.
new_df['Transaction_7-14'] = new_df['Transaction_7-14'].astype(int)

**15** Number of transitions made in the evening (15-22)

In [299]:
after_fifteen = df[df['TransactionTime'].dt.hour >= 15]
between_fifteen_twentytwo = after_fifteen[after_fifteen['TransactionTime'].dt.hour <= 22]
new_df['Transaction_15-22'] = between_fifteen_twentytwo.groupby(['CustomerID'])['TransactionTime'].count()
new_df['Transaction_15-22'] = new_df['Transaction_15-22'].fillna(0) #we replace NaN values, that is who didn't make transaction in summer, with 0.
new_df['Transaction_15-22'] = new_df['Transaction_15-22'].astype(int)